In [1]:
from PyPDF2 import PdfReader
import pdfplumber
import os
import re
import pandas as pd
import numpy as np

path = "data/Key_Stats/"
files = os.listdir(path)

#dir1 = path + files[178]

In [2]:
# tokenize the string with distance index
def tokenize(string):
    string = string.lower()
    # keep english and numbers with '%'

    string = re.sub(r'[^a-z0-9% ]', ' ', string)
    string = re.sub(r' +', ' ', string)
    # delete empty string
    string = string.strip()
    return string.split(' ')
'''
def read_pages(dir):
    text = ''
    pdf = PdfReader(dir)
    for page in pdf.pages:
        text += page.extract_text()
    return text
'''

def read_pages(dir):
    with pdfplumber.open(dir) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

def full_text(text):
    text = re.sub(r'[^\x00-\x7F]+',' ', text) # only english
    text = re.sub(r'[\n\t]+', ' ', text) # delete '\n' and '\t'
    text = re.sub(r' +', ' ', text) # delete multiple spaces
    return text

def file_id(file):
    pattern = r'[A-Z]{3,3}[0-9]{3,3}'
    return re.findall(pattern, file)

In [11]:
# divide the full text
section_dict = {'fact':['quick facts'], \
                'strategy':['objective and investment strategy', 'objectives and investment strategy'], \
                'derivative':['Use of derivatives / investment in derivatives', 'use of derivatives', 'investment in derivatives'], \
                'risk':['what are the key risks?', 'what are the key risk'], \
                'performance':['how has the sub-fund performed?','how has the sub -fund performed?', 'how has the fund performed'], \
                'guarantee':['is there any guarantee'], \
                'fees':['what are the fees and charges?', 'what are the fees?'], \
                'misc':['additional information']
                }



def division(text):
    section = {}
    for key in section_dict['fact']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['head'] = temp
            break
    for key in section_dict['strategy']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['fact'] = temp
            break
    for key in section_dict['derivative']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['strategy'] = temp
            break
    for key in section_dict['risk']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['derivative'] = temp
            break
    for key in section_dict['performance']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['risk'] = temp
            break
    for key in section_dict['guarantee']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['performance'] = temp
            break
    for key in section_dict['fees']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['guarantee'] = temp
            break
    for key in section_dict['misc']:
        if key in text:
            temp = text.split(key)[0]
            text = text.split(key)[1]
            section['fees'] = temp
            section['misc'] = text
            break
    return section

In [12]:
# build the corpus
corpus = pd.DataFrame(columns=['id', 'head', 'fact', 'strategy', 'derivative', 'risk', 'performance', 'guarantee', 'fees', 'misc'])

for i in range(len(files)):
    doc_dir = path + files[i]
    raw = read_pages(doc_dir)
    text = full_text(raw)
    doc_id = file_id(files[i])
    section = division(text.lower())
    data = pd.DataFrame({'id':doc_id, 'head':section['head'], 'fact':section['fact'], 'strategy':section['strategy'], 'derivative':section['derivative'], 'risk':section['risk'], 'performance':section['performance'], 'guarantee':section['guarantee'], 'fees':section['fees'], 'misc':section['misc']}, index=[i])
    corpus = pd.concat([corpus, data], axis=0)
    print(i)
    corpus.to_csv('data/corpus.csv', index=False)

0
1
2
3


KeyError: 'derivative'

In [13]:
i = 4
doc_dir = path + files[i]
raw = read_pages(doc_dir)
text = full_text(raw)
doc_id = file_id(files[i])
section = division(text.lower())
data = pd.DataFrame({'id':doc_id, 'head':section['head'], 'fact':section['fact'], 'strategy':section['strategy'], 'derivative':section['derivative'], 'risk':section['risk'], 'performance':section['performance'], 'guarantee':section['guarantee'], 'fees':section['fees'], 'misc':section['misc']}, index=[i])
corpus = pd.concat([corpus, data], axis=0)


KeyError: 'derivative'

In [16]:
text.lower().find('objectives and investment strategy')

2990

In [6]:
text[9200:9400]

'rket crashes or major crisis). Use of derivatives / investment in derivatives The Sub-Fund s net derivative exposure may be up to 50% of its Net Asset Value. What are the key risks? Investment involve'

In [17]:
section['strategy']

' objective the investment objective of the sub-fund is to provide income and capital appreciation through primarily investing in a portfolio of debt securities. there can be no assurance that the sub-fund will achieve its investment objective. strategy the sub-fund seeks to achieve its objective primarily through investing in debt securities. the sub-fund makes investments into debt securities that, in the manager s opinion, have been undervalued or mis-priced by the market. there is no restriction on the geographies, industries or credit ratings in relation to the debt securities in which the sub-fund may invest. the sub-fund is actively managed without reference to any benchmark. the manager will adopt a dynamic approach in asset allocation and security selection, based on a number of factors including macroeconomic analysis, issuer s credit analysis, and security valuation. the sub-fund may from time to time invest up to 100% of its net asset value in emerging markets, such as main